In [9]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time, string, re, nltk
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def get_page(driver, url):
    driver.get(url)
    like = driver.find_elements_by_class_name('js-abstract-parent')
    for x in range(0,len(like)):
        if like[x].is_displayed():
            like[x].click()
        time.sleep(1)
    time.sleep(4)
    driver.find_element_by_class_name("section-container")   
    return driver.page_source

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.minimize_window()
all_data = {"title":[], "author":[], "refrence":[], "volume":[], "issue":[], "date":[], "pages":[], "link":[], "abstract":[]}
for k in range(1, 29):
    for i in range(1, 7):
        print(f"Loading Data From Page {k}/28 Issue {i}/6 ......")
        url = "https://academic.oup.com/jamia/issue/"+str(k)+"/"+str(i)
        try:
            data = get_page(driver=driver, url=url)
            print(f"Page {k}/28 Issue {i}/6 Loaded Successfully.")
        except: print(f"Page {k}/28 Issue {i}/6 Did't Loaded.")
        soup = BeautifulSoup(data, 'lxml')
        article_data = soup.find_all("div", class_ = "al-article-items")
        for article in article_data:
            try:all_data['title'].append(article.find("a", class_ = "at-articleLink").text.replace('\n', ''))
            except:all_data['title'].append("")
            try: all_data["author"].append(article.find("div", class_ = "al-authors-list").text.replace('\n', ''))
            except: all_data["author"].append("")
            try:refrence, volume, issue, date, pages, link = article.find("div", class_ = "ww-citation-primary").text.split(',')
            except:refrence, volume, issue, date, pages, link ="", "", "", "", "", ""
            try:abstract = article.find("section", class_ = "abstract").text
            except: abstract=''
            all_data['refrence'].append(refrence)
            all_data["volume"].append(volume)
            all_data["issue"].append(issue)
            all_data["date"].append(date)
            all_data["pages"].append(pages)
            all_data["link"].append(link)
            all_data["abstract"].append(abstract)



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\Administrator\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache


Loading Data From Page 1/28 Issue 1/6 ......
Page 1/28 Issue 1/6 Loaded Successfully.
Loading Data From Page 1/28 Issue 2/6 ......
Page 1/28 Issue 2/6 Loaded Successfully.
Loading Data From Page 1/28 Issue 3/6 ......
Page 1/28 Issue 3/6 Loaded Successfully.
Loading Data From Page 1/28 Issue 4/6 ......
Page 1/28 Issue 4/6 Loaded Successfully.
Loading Data From Page 1/28 Issue 5/6 ......
Page 1/28 Issue 5/6 Loaded Successfully.
Loading Data From Page 1/28 Issue 6/6 ......
Page 1/28 Issue 6/6 Loaded Successfully.
Loading Data From Page 2/28 Issue 1/6 ......
Page 2/28 Issue 1/6 Loaded Successfully.
Loading Data From Page 2/28 Issue 2/6 ......
Page 2/28 Issue 2/6 Loaded Successfully.
Loading Data From Page 2/28 Issue 3/6 ......
Page 2/28 Issue 3/6 Loaded Successfully.
Loading Data From Page 2/28 Issue 4/6 ......
Page 2/28 Issue 4/6 Loaded Successfully.
Loading Data From Page 2/28 Issue 5/6 ......
Page 2/28 Issue 5/6 Loaded Successfully.
Loading Data From Page 2/28 Issue 6/6 ......
Page 2/28

In [5]:
data_frame = pd.DataFrame()
data_frame = data_frame.from_dict(all_data)

In [11]:
stopwords_list = stopwords.words('english') 
punctuations_list = string.punctuation 
stemmer = PorterStemmer()
def text_cleaner(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower())
    text = text + " ".join(emoticons).replace('-', '')
    return " ".join([stemmer.stem(word) for word in nltk.tokenize.word_tokenize(text) if (word not in stopwords_list) and (word not in punctuations_list) and (word.isalpha())])

data_frame["Clean Abstract"] = data_frame["abstract"].apply(text_cleaner) 

In [12]:
data_frame.to_csv("JAMIA.CSV", index=False)